## A stateless oracle: introduction
#### 09.0 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-03-24

* Part 0: Theoretical introduction
* Parts 1-4 are only relevant if you want to **create** an Oracle
* Parts 5-6 are needed to **use** the oracle.

## Introduction
The distincion between "stateless" and "stateful" smart contracts is often defined in such a way that
* Stateless Smart Contracts only evaluate the merits of a proposed transaction (without interacting with the blockchain), while
* Stateful Smart Contracts fully interact with the blockchain

Strictly speaking, however, **there are no stateless payment transactions**. Every payment transaction must be checked against the blockchain against a possible overspend, which requires a look-up on the blockchain.

In the following, we will show how this fact can be used to create an oracle that is based entirely on "stateless" smart contracts.

## Goal
The goal of this chapter is to create an oracle that is completely on chain and does not use stateful smart contracts. The principle will be illustrated with an oracle for the ALGO-USD exchange rate, but the principle is not limited to exchange rate or trading assets.

## In a nutshell
1. We create an Oracle coin and two accounts called `Price` and `Reserve`. 
1. The (external) oracle will rebalance the holdings of the account such that **the holdings of `Price` will reflect the USD/ALGO exchange rate**.
2. We then create one Smart Signature for each `Price` and `Reserve` such that these accounts can transfer some or all of their coins exclusively to *themselves*.
3. We  then create a Smart Signature for an exchange that uses the oracle to authorize atomic swaps that correctly reflect the exchange rate.

## Framework

While a stateless contract (or transaction group of stateless contracts) has no access to the holdings of an address, all individual transaction amounts are available to all transactions in the transaction group. **The goal is hence to find a way to encode the holdings of an address in the form of a transaction amount.**

Capital letters denote holdings and lower case letters denote amounts in asset transfer transactions.

1. The supply of an ASA is fixed and known without referring to the blockchain, formally $$T = const$$
2. If the creator of the ASA transfers all coins exclusively to two trusted accounts $P$ ("price") and $R$ ("reserve"), which are governed by smart contracts that only allow (a) transfers to oneself or (b) transfers to the other holding account, then their holdings add up to the total, fomally $$P + R = T$$
3. Any payment amount must be smaller or equal to the total holding of the payer, formally $$p \leq P$$ and $$r \leq R$$
4. The total amount transferred in one transaction group is hence $$p +r \leq P + R = T$$
5. If we now require (remembering $T=const$) $$p +r  = T,$$ then the inequalities become equalities and we have $$p =P$$ and $$r=R$$
6. Hence the transaction amount $p$ reflects the holdings $P$. $\square$

**Remark:** Condition (5.) also ensures against a potential malicious actor $S$ obtaining some orscle coins.

#### The Smart Signature for the oracle
The third line of the oracle_condition ensures that $p+r = T$

In [ ]:
oracle_condition = And (
    Gtxn[0].sender() == Addr(price_sig['public_addr']),                             # p 
    Gtxn[1].sender() == Addr(reserve_sig['public_addr']),                           # r
    Gtxn[0].asset_amount() + Gtxn[1].asset_amount() == Int(int( 1e3 * 1e6 ))        # p + r = T (small units)
    ) 
# safety conditions omitted for simplicity, see 09.6 for entire smart signature

#### Using the oracle
To use the oracle in an atomic swap, we need a transaction group of 4 transactions:
* `Txn[0]` a transaction of the *Price* account with itself to obtain the price
* `Txn[1]` a transaction of the *Reserve* account to verify the price
    * Criterion: the amounts of `Txn[0]` and `Txn[1]` must add up exactly to the total supply of Oracle Coins
* `Txn[2]` the ALGO transaction
* `Txn[3]` the USDC transaction
    * Criterion: the amounts of `Txn[2]` and `Txn[3]` must correctly reflect the exchange rate
    * The exchange rate is obtained from `Txn[0]` 

**Note** transactions 0 and 1 are the oracle part, transactions 2 and 3 are the actual atomic swap




In [ ]:
oracle_condition = And (
    Gtxn[0].sender() == Addr(price_sig['public_addr']),                             # p 
    Gtxn[1].sender() == Addr(reserve_sig['public_addr']),                           # r
    Gtxn[0].asset_amount() + Gtxn[1].asset_amount() == Int(int( 1e3 * 1e6 ))        # p + r = T (small units)
    ) 

exchange_condition = And (
    Gtxn[2].xfer_asset() == Int(0),                    # Txn2 is in ALGOs
    Gtxn[3].xfer_asset() == Int(USDC_id),              # Txn3 is in USDC
    # Exchange rate in small units (note: Algo, Oracle and USDC *all* have 6 decimals)
    # Exchange rate is taken from Gtxn[0].asset_amount()
    # ALGO_amount * USD_per_ALGO == USD_amount
    Gtxn[2].amount() * Gtxn[0].asset_amount() / Int(int(1e6)) == Gtxn[3].asset_amount(),
)

# safety conditions omitted for simplicity, see 09.6 for entire smart signature
exchange_pyteal = And(
    exchange_condition,
    oracle_condition,
    safety_condition
    )

## Example
A working example of the Oracle Coin, reflecting the USD/ALGO exchange rate can be found on the testnet:
https://testnet.algoexplorer.io/asset/77534697